In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os
import glob
from time import sleep
import json
from datetime import date 
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
spot = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id = client_id,
                                                                           client_secret= client_secret))


In [2]:
class Band:
    def __init__(self,name: str):
        data_band = spot.search(name,type = 'artist',market='BR')['artists']['items']
        self.name = data_band[0]['name']
        self.id = data_band[0]['id']

class Albuns:
    def __init__(self,band_id: str):
        #self.data=  spot.artist_albums(band_id,album_type='album', country='BR')['items']
        self.id = band_id
        albuns = []
        for album in spot.artist_albums(band_id,country='BR')['items']:
            albuns.append((album['name'],album['id'],album['album_group']))
        self.albuns = albuns

class Album:
    def __init__(self,album_id: str):
        self.id = album_id
        data_album = spot.album(album_id,market='BR')
        self.band = data_album['artists'][0]['name']
        self.name = data_album['name']
        self.popularity = data_album['popularity']
        self.release_date = data_album['release_date']
        self.total_tracks = data_album['total_tracks']
                
class Tracks_in_Album:
    
    def __init__(self, album_id: str):
        #self.data = spot.album_tracks(album_id, market ='BR')['items']
        self.id = album_id
        tracks = []
        for track in spot.album_tracks(album_id,market='BR')['items']:
            tracks.append((track['name'],track['id']))
        self.tracks = tracks
            
class Track:
    def __init__ (self, track_id:str):
#         self.data = spot.track(track_id,market='BR')
        data_track = spot.track(track_id,market='BR')
        self.id = track_id
        self.band = data_track['artists'][0]['name']
        self.album = data_track['album']['name']
        self.name = data_track['name']
        self.popularity = data_track['popularity']
        data_features = spot.audio_features(track_id)
        self.danceability = data_features[0]['danceability']
        self.energy = data_features[0]['energy']
        self.loudness = data_features[0]['loudness']
        self.mode = data_features[0]['mode']
        self.speechiness = data_features[0]['speechiness']
        self.acousticness = data_features[0]['acousticness']
        self.instrumentalness = data_features[0]['instrumentalness']
        self.liveness = data_features[0]['liveness']
        self.valence = data_features[0]['valence']
        self.tempo = data_features[0]['tempo']
        self.duration_ms = data_features[0]['duration_ms']
        self.time_signature = data_features[0]['time_signature']
        
        
class TopTracks:
    def __init__(self,band_id:str):
        data_toptracks = spot.artist_top_tracks(band_id)['tracks']
        tracks = []
        for track in data_toptracks:
            tracks.append((track['name'],track['id']))
        self.tracks = tracks

In [3]:
def all_songs(artist):
    if isinstance(artist,str):
        lista_de_bandas = [artist]
    if isinstance(artist,list):
        lista_de_bandas = artist
    import_tracks = []
    import_albums = []
    df_songs = []
    df_albums = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_albums = Albuns(banda.id)

        for album in banda_albums.albuns:
#             di_a = vars(Album(album[1]))
#             di_a['album_group'] = album[2]
#             di_a['date_import'] = date.today()
#             print(di_a)

            for musica in Tracks_in_Album(album[1]).tracks:
#                 print(musica[0])
                di_s = vars(Track(musica[1]))
                di_s['date_import'] = date.today()
                import_tracks.append(di_s)
    df_songs = pd.DataFrame(import_tracks)
    df_songs.drop_duplicates(inplace=True)
    return df_songs

def all_albuns(artist):
    lista_de_bandas = [artist]
    import_albums = []
    df_albums = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_albums = Albuns(banda.id)

        for album in banda_albums.albuns:
            di_a = vars(Album(album[1]))
            di_a['album_group'] = album[2]
            di_a['date_import'] = date.today()
#             print(di_a)
            import_albums.append(di_a)
#             for musica in Tracks_in_Album(album[1]).tracks:
#                 print(musica[0])

    df_albums = pd.DataFrame(import_albums)
    df_albums.drop_duplicates(inplace=True) 
    return df_albums

def top_tracks(artist):
    posicao = 1
    lista_de_bandas = [artist]
    import_tracks = []
    df_songs = []
    for banda1 in lista_de_bandas:
        banda = Band(banda1)
        banda_toptracks = TopTracks(banda.id)

#         print(f'Importando: {banda.name}')
        for musica in banda_toptracks.tracks:
    #         print(musica[0])
            di_s = vars(Track(musica[1]))
            di_s['date_import'] = date.today()
            di_s['position'] = posicao
            import_tracks.append(di_s)
            posicao += 1
        posicao = 1
    df_songs = pd.DataFrame(import_tracks)
    print('Finalizado')
    return df_songs

def playlist(playlist_id):
    position = 1
    lista_dict = []
    for track in spot.playlist(playlist_id)['tracks']['items']:
        dict1 = {}
        lista_artista =[]
        for artista in track['track']['artists']:
            lista_artista.append(artista['name'])
        dict1['position'] = position
        dict1['song'] = track['track']['name']
        dict1['artists'] = lista_artista
        dict1['date_import'] = today = date.today()
        position += 1
        lista_dict.append(dict1)
    df_playlist = pd.DataFrame(lista_dict)
    df_playlist.set_index('position',inplace=True)
    return df_playlist

In [4]:
position = 1
lista_dict = []
for track in spot.playlist('37i9dQZEVXbMXbN3EUUhlg')['tracks']['items']:
    dict1 = {}
    lista_artista =[]
    for artista in track['track']['artists']:
        lista_artista.append(artista['name'])
    dict1['position'] = position
    dict1['song'] = track['track']['name']
    dict1['artists'] = lista_artista
    dict1['date_import'] = today = date.today()
    dict1.update(vars(Track(track['track']['id'])))
    
    position += 1
    lista_dict.append(dict1)
df_playlist = pd.DataFrame(lista_dict)
df_playlist.drop(labels='name',axis=1, inplace=True)

In [5]:
df_playlist['danceability'] = df_playlist['danceability']*100
df_playlist['energy'] = df_playlist['energy']*100
df_playlist['valence'] = df_playlist['valence']*100
df_playlist['acousticness'] = df_playlist['acousticness']*100



df_melt = df_playlist.melt(id_vars=['position','song','artists','id','band','album'], value_vars=['popularity','danceability','energy','valence','acousticness'],var_name = 'features' )

In [7]:
df_melt.to_csv('top_50_melt.csv',sep=';')